In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
import xgboost as xgb
from sklearn.model_selection import cross_val_score, train_test_split
import pandas as pd

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
df = pd.read_csv('competition_table.csv',sep=';',index_col=0)
# replace commas for dots and convert previous string column into float
def processCol(col):
    try:
        return col.astype(str).apply(lambda val: val.replace(',','.')).astype(float)
    except:
        return col


df = df.apply(processCol)
#print(len(df))
#keep_seasons = ['2019-2020','2020-2021','2021-2022']
#df = df[df.season.isin(keep_seasons)]

print(len(df))
df.head()

64166


,season,div,odds_home_team_win,odds_draw,odds_away_team_win,home_team_match_nr,home_team_goal_roll1_sum,home_team_goal_roll2_sum,home_team_goal_roll3_sum,home_team_goal_roll4_sum,...,odds_away_team_draw_roll4_mean,odds_away_team_defeat_roll1_mean,odds_away_team_defeat_roll2_mean,odds_away_team_defeat_roll3_mean,odds_away_team_defeat_roll4_mean,away_team_point_mean,away_team_expected_point_mean,home_win_flag,draw_flag,away_win_flag
match_id,,,,,,,,,,,,,,,,,,,,,
0,2008-2009,div0,1.73,3.5,5.00,5.0,3.0,7.0,8.0,10.0,...,3.825,11.00,7.665,6.377,5.908,0.500,3.0,0.0,0.0,1.0
1,2008-2009,div0,1.25,5.5,12.00,7.0,0.0,2.0,5.0,9.0,...,3.582,2.10,2.250,2.833,2.475,0.333,0.0,0.0,1.0,0.0
2,2008-2009,div0,1.57,3.8,5.75,9.0,1.0,4.0,4.0,6.0,...,3.525,6.75,4.275,3.883,3.638,0.750,1.5,1.0,0.0,0.0
3,2008-2009,div0,1.17,7.0,15.00,11.0,2.0,6.0,7.0,10.0,...,3.975,4.20,2.750,2.333,2.550,0.600,0.0,1.0,0.0,0.0
4,2008-2009,div0,1.17,7.0,15.00,14.0,2.0,4.0,7.0,9.0,...,3.925,3.10,2.175,2.050,2.015,0.833,0.0,1.0,0.0,0.0


In [7]:
def base_model(df,target_col):
    #data prep
    features_cols = [x for x in df.columns if x not in ['home_win_flag', 'draw_flag', 'away_win_flag', 'div','season']]
    #features_cols = [x for x in features_cols if x.find('odds') == -1]
    #features_col = ['odds_home_team_win',  'odds_away_team_win']
    if target_col == 'draw_flag':
        random_state = 12
    elif target_col == 'home_win_flag': 
        random_state = 456
    else:
        random_state = 4562
        
    random_state = 1
    X_train, X_test, y_train, y_test = train_test_split(df[features_cols], df[target_col], test_size=0.3, random_state=random_state)
       
    
    if target_col != 'draw_flag':
        df_without_draw = df[df.draw_flag==0]
        
    else:
        df_without_draw = df

    # Split data into train and test sets
    X_train, X_test__, y_train, y_test__ = train_test_split(df_without_draw[features_cols], df_without_draw[target_col], test_size=0.3, random_state=random_state)
    #Pipeline
    pipeline = make_pipeline(StandardScaler(), LogisticRegression(penalty='l2', solver='liblinear', max_iter=10000))
    # Fit pipeline to training set and evaluate on test set
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict_proba(X_test)[:, 1]

    # Calculate false positive rate, true positive rate, and area under curve (AUC)
    fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(fpr, tpr)
    print(f" { target_col } auc is: {roc_auc}.")
    #pipeline.named_steps['logisticregression']
    return pipeline

In [8]:
def create_target_columns_to_classification(df):
    target_cols = ['home_win_flag', 'draw_flag','away_win_flag']
    target_cols_dict = {'home_win_flag':0,'draw_flag':1,'away_win_flag':2}
    def get_y(row):
        for col in target_cols:
            if row[col] == 1:

                y = target_cols_dict[col]
        return y
    df['y'] = df[target_cols].apply(get_y,axis=1)
    df['y'] = df['y'].astype('category')
    return df

In [9]:
p_home_win = base_model(df,'home_win_flag')
p_draw = base_model(df,'draw_flag')
p_away_win = base_model(df,'away_win_flag')

 home_win_flag auc is: 0.6840293532186246.
 draw_flag auc is: 0.553859924387211.
 away_win_flag auc is: 0.6912572686683559.


In [10]:
target_col = 'home_win_flag'
features_cols = [x for x in df.columns if x not in ['home_win_flag', 'draw_flag', 'away_win_flag', 'div','season']]
#features_cols = [x for x in features_cols if x.find('odds') == -1]
target_cols = ['home_win_flag','draw_flag','away_win_flag']

#features_col = ['odds_home_team_win',  'odds_away_team_win']
X_train, X_test, y_train, y_test = train_test_split(df[features_cols], df[target_col], test_size=0.3, random_state=23)
print(f'train data length is {len(y_train)}')
print(f'test data length is {len(y_test)}')
val_df = pd.DataFrame(y_test)
val_df.columns = ['home_win_flag']
val_df['draw_flag'] = df.loc[y_test.index.values]['draw_flag']
val_df['away_win_flag'] = df.loc[y_test.index.values]['away_win_flag']
val_df['home_win_flag_base_pred'] = p_home_win.predict(X_test)
val_df['draw_flag_base_pred'] = p_draw.predict(X_test)
val_df['away_win_flag_base_pred'] = p_away_win.predict(X_test)
val_df = create_target_columns_to_classification(val_df)
val_df[['home_win_prob_0','home_win_prob_1']] = p_home_win.predict_proba(X_test)
val_df[['draw_prob_0','draw_prob_1']] = p_draw.predict_proba(X_test)
val_df[['away_win_prob_0','away_win_prob_1']] = p_away_win.predict_proba(X_test)
for y in target_cols:    
    fpr, tpr, thresholds = roc_curve(val_df[y],val_df[y+'_base_pred'])
    print(f' {y} auc is: {auc(fpr, tpr)}')

train data length is 44916
test data length is 19250
 home_win_flag auc is: 0.6015239125693428
 draw_flag auc is: 0.5
 away_win_flag auc is: 0.619568882876796


In [11]:
retrain_index = val_df[(val_df.home_win_flag_base_pred != val_df.away_win_flag)].index.values
len(retrain_index)

13318

In [12]:
def create_XGBoost_model(models_output_df_features,target):
    from sklearn.model_selection import GridSearchCV
    # Set XGBoost parameters
    param = {
        'max_depth': [2,4,10],
        'learning_rate': [0.1],
        'subsample': [0.8],
        'colsample_bytree': [0.8],
        'n_estimators': [100, 500]
    }

    # Create a GridSearchCV object and fit it to the training data
    xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=3, n_jobs=-1)
    grid_search = GridSearchCV(xgb_model, param_grid=param, cv=3, verbose=3, n_jobs=-1)
    grid_search.fit(models_output_df_features, target)

    # Print the best parameters and the corresponding score
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best score: {grid_search.best_score_}")

    # Predict on test data with the best estimator from grid search
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(models_output_df_features)
    y_test = target
    # Calculate accuracy
    accuracy = sum(y_test == y_pred) / len(y_test)
    print(f"Accuracy: {accuracy}")
    return best_model

In [13]:
target_col = 'home_win_flag'
features_cols = [x for x in df.columns if x not in ['home_win_flag', 'draw_flag', 'away_win_flag', 'div','season']]
#features_cols = [x for x in features_cols if x.find('odds') == -1]

retrain_df = df.loc[retrain_index]
print(len(retrain_df))
retrain_df = create_target_columns_to_classification(retrain_df)



13318


In [14]:
xgb_m = create_XGBoost_model(retrain_df[features_cols],retrain_df['y'])

Fitting 3 folds for each of 6 candidates, totalling 18 fits
Best parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 100, 'subsample': 0.8}
Best score: 0.6945488628975901
Accuracy: 0.7034840066075987


In [15]:
val_df.loc[retrain_index,'final_pred']  = xgb_m.predict(retrain_df[features_cols])
val_df

,home_win_flag,draw_flag,away_win_flag,home_win_flag_base_pred,draw_flag_base_pred,away_win_flag_base_pred,y,home_win_prob_0,home_win_prob_1,draw_prob_0,draw_prob_1,away_win_prob_0,away_win_prob_1,final_pred
match_id,,,,,,,,,,,,,,
56856,0.0,0.0,1.0,0.0,0.0,1.0,2,0.652614,0.347386,0.727404,0.272596,0.347385,0.652615,2.0
31414,0.0,0.0,1.0,0.0,0.0,1.0,2,0.720167,0.279833,0.708755,0.291245,0.279837,0.720163,2.0
31725,1.0,0.0,0.0,1.0,0.0,0.0,0,0.454646,0.545354,0.698274,0.301726,0.545350,0.454650,0.0
57922,0.0,1.0,0.0,0.0,0.0,1.0,1,0.558104,0.441896,0.767066,0.232934,0.441912,0.558088,NaN
32239,0.0,0.0,1.0,1.0,0.0,0.0,2,0.357675,0.642325,0.715948,0.284052,0.642320,0.357680,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37661,1.0,0.0,0.0,1.0,0.0,0.0,0,0.193517,0.806483,0.808437,0.191563,0.806484,0.193516,0.0
15556,1.0,0.0,0.0,1.0,0.0,0.0,0,0.157610,0.842390,0.770519,0.229481,0.842392,0.157608,0.0
21217,0.0,1.0,0.0,1.0,0.0,0.0,1,0.321028,0.678972,0.745512,0.254488,0.678968,0.321032,0.0


In [16]:
val_df['final_pred'] = np.where(pd.isnull(val_df['final_pred']), val_df['y'], val_df['final_pred'],)

In [17]:
# Ok, itt tultanitom 

In [18]:
def get_pred_targets(x,colname):
    value = target_cols_dict[colname]
    a = 0
    if value == x:
        a = 1
    
    return a
target_cols_dict = {'home_win_flag':0,'draw_flag':1,'away_win_flag':2}
target_cols = ['home_win_flag','draw_flag','away_win_flag']
for t in target_cols:
    val_df[t+'_pred'] = val_df['final_pred'].apply(lambda x: get_pred_targets(x,t))
for y in target_cols:    
    fpr, tpr, thresholds = roc_curve(val_df[y],val_df[y+'_pred'])
    print(f' {y} auc is: {auc(fpr, tpr)}')

 home_win_flag auc is: 0.8197659448770052
 draw_flag auc is: 0.6433269062669519
 away_win_flag auc is: 0.9710207376131027


In [19]:
#lets validate 

In [20]:
df = pd.read_csv('competition_table.csv',sep=';',index_col=0)
# replace commas for dots and convert previous string column into float
def processCol(col):
    try:
        return col.astype(str).apply(lambda val: val.replace(',','.')).astype(float)
    except:
        return col


df = df.apply(processCol)
#keep_seasons = ['2019-2020','2020-2021','2021-2022']
#df = df[df.season.isin(keep_seasons)]


df.head()

target_col = 'home_win_flag'
features_cols = [x for x in df.columns if x not in ['home_win_flag', 'draw_flag', 'away_win_flag', 'div','season']]
#features_cols = [x for x in features_cols if x.find('odds') == -1]
target_cols = ['home_win_flag','draw_flag','away_win_flag']

#features_col = ['odds_home_team_win',  'odds_away_team_win']
X_train, X_test, y_train, y_test = train_test_split(df[features_cols], df[target_col], test_size=0.3, random_state=2222)
val_df = pd.DataFrame(y_test)
val_df.columns = ['home_win_flag']
val_df['draw_flag'] = df.loc[y_test.index.values]['draw_flag']
val_df['away_win_flag'] = df.loc[y_test.index.values]['away_win_flag']
val_df['home_win_flag_base_pred'] = p_home_win.predict(X_test)
val_df['draw_flag_base_pred'] = p_draw.predict(X_test)
val_df['away_win_flag_base_pred'] = p_away_win.predict(X_test)
val_df = create_target_columns_to_classification(val_df)
retrain_index = val_df[(val_df.home_win_flag_base_pred != val_df.away_win_flag)].index.values
val_df.loc[retrain_index,'final_pred']  = xgb_m.predict(X_test.loc[retrain_index][features_cols])
val_df['final_pred'] = np.where(pd.isnull(val_df['final_pred']), val_df['y'], val_df['final_pred'],)
for t in target_cols:
    val_df[t+'_pred'] = val_df['final_pred'].apply(lambda x: get_pred_targets(x,t))
for y in target_cols:    
    fpr, tpr, thresholds = roc_curve(val_df[y],val_df[y+'_pred'])
    print(f' {y} auc is: {auc(fpr, tpr)}')

 home_win_flag auc is: 0.8231605758388142
 draw_flag auc is: 0.6461899226337326
 away_win_flag auc is: 0.9717981959949795


In [21]:
import joblib
import pickle

In [23]:
# save model
pickle.dump(p_home_win, open('pipe_home_win_base_model_3.p', "wb"))
pickle.dump(p_draw, open('pipe_p_draw_base_model_3.p', "wb"))
pickle.dump(p_away_win, open('pipe_p_away_win_base_model_3.p', "wb"))
pickle.dump(xgb_m, open('xgb_m_3.p', "wb"))